In [135]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter

In [136]:
# 1. Загрузка данных
data = pd.read_csv('movie_bd_v5.csv')

# 2. Просмотр состояния данных
data.sample(5)

# Проверяем есть ли пропущенные значения
data.isnull().any().any()

# Просмотр первых элементов данных
data.head()

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year
0,tt0369610,150000000,1513528810,Jurassic World,Chris Pratt|Bryce Dallas Howard|Irrfan Khan|Vi...,Colin Trevorrow,The park is open.,Twenty-two years after the events of Jurassic ...,124,Action|Adventure|Science Fiction|Thriller,Universal Studios|Amblin Entertainment|Legenda...,6/9/2015,6.5,2015
1,tt1392190,150000000,378436354,Mad Max: Fury Road,Tom Hardy|Charlize Theron|Hugh Keays-Byrne|Nic...,George Miller,What a Lovely Day.,An apocalyptic story set in the furthest reach...,120,Action|Adventure|Science Fiction|Thriller,Village Roadshow Pictures|Kennedy Miller Produ...,5/13/2015,7.1,2015
2,tt2908446,110000000,295238201,Insurgent,Shailene Woodley|Theo James|Kate Winslet|Ansel...,Robert Schwentke,One Choice Can Destroy You,Beatrice Prior must confront her inner demons ...,119,Adventure|Science Fiction|Thriller,Summit Entertainment|Mandeville Films|Red Wago...,3/18/2015,6.3,2015
3,tt2488496,200000000,2068178225,Star Wars: The Force Awakens,Harrison Ford|Mark Hamill|Carrie Fisher|Adam D...,J.J. Abrams,Every generation has a story.,Thirty years after defeating the Galactic Empi...,136,Action|Adventure|Science Fiction|Fantasy,Lucasfilm|Truenorth Productions|Bad Robot,12/15/2015,7.5,2015
4,tt2820852,190000000,1506249360,Furious 7,Vin Diesel|Paul Walker|Jason Statham|Michelle ...,James Wan,Vengeance Hits Home,Deckard Shaw seeks revenge against Dominic Tor...,137,Action|Crime|Thriller,Universal Pictures|Original Film|Media Rights ...,4/1/2015,7.3,2015


In [137]:
data.describe()

,budget,revenue,runtime,vote_average,release_year
count,1.889000e+03,1.889000e+03,1889.000000,1889.000000,1889.000000
mean,5.431083e+07,1.553653e+08,109.658549,6.140762,2007.860773
std,4.858721e+07,2.146698e+08,18.017041,0.764763,4.468841
min,5.000000e+06,2.033165e+06,63.000000,3.300000,2000.000000
25%,2.000000e+07,3.456058e+07,97.000000,5.600000,2004.000000
50%,3.800000e+07,8.361541e+07,107.000000,6.100000,2008.000000
75%,7.200000e+07,1.782626e+08,120.000000,6.600000,2012.000000
max,3.800000e+08,2.781506e+09,214.000000,8.100000,2015.000000


# Предобработка

In [138]:
answers = {} # создадим словарь для ответов

# Создаем колонку с данными по прибыли "profit"
data['profit'] = data['revenue']-data['budget']

# Создаем колонку со списком жанров, в текущей версии данных все жанры по ленте объединены в одну строку с разделителем "|"
data['genre_list'] = data.genres.apply(lambda genres: genres.split('|'))

# Создаем колонку со списком директоров по каждой ленте, так как есть
    # фильмы у которых несколько директоров и записаны они строкой с разделителем "|"
data['director_list'] = data.director.apply(lambda directors: directors.split('|'))

# Создаем колонку со списком актеров по каждой ленте
data['cast_list'] = data.cast.apply(lambda cast: cast.split('|'))

# Создаем колонку с длиной названия киноленты
data['tittle_width'] = data.original_title.apply(lambda title: len(title))

# Создаем колонку с названиями кинокомпаний
data['production_company_list'] = data.production_companies.apply(lambda production_company: production_company.split('|'))

# Создаем колонку с количеством слов в описании к фильму
data['description_width'] = data.overview.apply(lambda text: len(text.split(' ')))

data.head()

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,...,release_date,vote_average,release_year,profit,genre_list,director_list,cast_list,tittle_width,production_company_list,description_width
0,tt0369610,150000000,1513528810,Jurassic World,Chris Pratt|Bryce Dallas Howard|Irrfan Khan|Vi...,Colin Trevorrow,The park is open.,Twenty-two years after the events of Jurassic ...,124,Action|Adventure|Science Fiction|Thriller,...,6/9/2015,6.5,2015,1363528810,"[Action, Adventure, Science Fiction, Thriller]",[Colin Trevorrow],"[Chris Pratt, Bryce Dallas Howard, Irrfan Khan...",14,"[Universal Studios, Amblin Entertainment, Lege...",26
1,tt1392190,150000000,378436354,Mad Max: Fury Road,Tom Hardy|Charlize Theron|Hugh Keays-Byrne|Nic...,George Miller,What a Lovely Day.,An apocalyptic story set in the furthest reach...,120,Action|Adventure|Science Fiction|Thriller,...,5/13/2015,7.1,2015,228436354,"[Action, Adventure, Science Fiction, Thriller]",[George Miller],"[Tom Hardy, Charlize Theron, Hugh Keays-Byrne,...",18,"[Village Roadshow Pictures, Kennedy Miller Pro...",110
2,tt2908446,110000000,295238201,Insurgent,Shailene Woodley|Theo James|Kate Winslet|Ansel...,Robert Schwentke,One Choice Can Destroy You,Beatrice Prior must confront her inner demons ...,119,Adventure|Science Fiction|Thriller,...,3/18/2015,6.3,2015,185238201,"[Adventure, Science Fiction, Thriller]",[Robert Schwentke],"[Shailene Woodley, Theo James, Kate Winslet, A...",9,"[Summit Entertainment, Mandeville Films, Red W...",22
3,tt2488496,200000000,2068178225,Star Wars: The Force Awakens,Harrison Ford|Mark Hamill|Carrie Fisher|Adam D...,J.J. Abrams,Every generation has a story.,Thirty years after defeating the Galactic Empi...,136,Action|Adventure|Science Fiction|Fantasy,...,12/15/2015,7.5,2015,1868178225,"[Action, Adventure, Science Fiction, Fantasy]",[J.J. Abrams],"[Harrison Ford, Mark Hamill, Carrie Fisher, Ad...",28,"[Lucasfilm, Truenorth Productions, Bad Robot]",26
4,tt2820852,190000000,1506249360,Furious 7,Vin Diesel|Paul Walker|Jason Statham|Michelle ...,James Wan,Vengeance Hits Home,Deckard Shaw seeks revenge against Dominic Tor...,137,Action|Crime|Thriller,...,4/1/2015,7.3,2015,1316249360,"[Action, Crime, Thriller]",[James Wan],"[Vin Diesel, Paul Walker, Jason Statham, Miche...",9,"[Universal Pictures, Original Film, Media Righ...",14


# 1. У какого фильма из списка самый большой бюджет?

Использовать варианты ответов в коде решения запрещено.    
Вы думаете и в жизни у вас будут варианты ответов?)

In [139]:
# Ответ на вопрос № 1. Верно "+"
answers['1'] = '5. Pirates of the Caribbean: On Stranger Tides (tt1298650)'

In [140]:
data.loc[data.budget == data.budget.max()][['imdb_id', 'original_title', 'budget']]

,imdb_id,original_title,budget
723,tt1298650,Pirates of the Caribbean: On Stranger Tides,380000000


ВАРИАНТ 2

In [141]:
# Вариант 2. Фильтрация через "query"
data.query('budget == {}'.format(data.budget.max()))

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,...,release_date,vote_average,release_year,profit,genre_list,director_list,cast_list,tittle_width,production_company_list,description_width
723,tt1298650,380000000,1021683000,Pirates of the Caribbean: On Stranger Tides,Johnny Depp|PenÃ©lope Cruz|Geoffrey Rush|Ian M...,Rob Marshall,Live Forever Or Die Trying.,Captain Jack Sparrow crosses paths with a woma...,136,Adventure|Action|Fantasy,...,5/11/2011,6.3,2011,641683000,"[Adventure, Action, Fantasy]",[Rob Marshall],"[Johnny Depp, PenÃ©lope Cruz, Geoffrey Rush, I...",43,"[Walt Disney Pictures, Jerry Bruckheimer Films...",75


# 2. Какой из фильмов самый длительный (в минутах)?

In [142]:
# Ответ на вопрос № 2. Верно "+"
answers['2'] = '2. Gods and Generals (tt0279111)'

In [143]:
data.loc[data.runtime == data.runtime.max()][['original_title', 'imdb_id', 'runtime']]

,original_title,imdb_id,runtime
1157,Gods and Generals,tt0279111,214


ВАРИАНТ 2

In [144]:
# Вариант 2. Фильтрация через "query"
data.query('runtime == {}'.format(data.runtime.max()))

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,...,release_date,vote_average,release_year,profit,genre_list,director_list,cast_list,tittle_width,production_company_list,description_width
1157,tt0279111,56000000,12923936,Gods and Generals,Stephen Lang|Jeff Daniels|Robert Duvall|Kevin ...,Ronald F. Maxwell,The nations heart was touched by...,The film centers mostly around the personal an...,214,Drama|History|War,...,2/21/2003,5.8,2003,-43076064,"[Drama, History, War]",[Ronald F. Maxwell],"[Stephen Lang, Jeff Daniels, Robert Duvall, Ke...",17,"[Turner Pictures, Antietam Filmworks]",48


# 3. Какой из фильмов самый короткий (в минутах)?





In [145]:
# Ответ на вопрос № 3. Верно "+"
answers['3'] = '3. Winnie the Pooh (tt1449283)'

In [146]:
data.loc[data.runtime == data.runtime.min()][['original_title', 'imdb_id', 'runtime']]

,original_title,imdb_id,runtime
768,Winnie the Pooh,tt1449283,63


ВАРИАНТ 2

In [147]:
# Вариант 2. Фильтрация через "query"
data.query('runtime == {}'.format(data.runtime.min()))

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,...,release_date,vote_average,release_year,profit,genre_list,director_list,cast_list,tittle_width,production_company_list,description_width
768,tt1449283,30000000,14460000,Winnie the Pooh,Jim Cummings|Travis Oates|Jim Cummings|Bud Luc...,Stephen Anderson|Don Hall,Oh Pooh.,"During an ordinary day in Hundred Acre Wood, W...",63,Animation|Family,...,4/13/2011,6.8,2011,-15540000,"[Animation, Family]","[Stephen Anderson, Don Hall]","[Jim Cummings, Travis Oates, Jim Cummings, Bud...",15,"[Walt Disney Pictures, Walt Disney Animation S...",52


# 4. Какова средняя длительность фильмов?


In [148]:
# Ответ на вопрос № 4. Верно "+"
answers['4'] = '2. 110'

In [149]:
data.runtime.mean()

109.6585494970884

In [214]:
# Округлим результат до целых минут

def int_r(num):
    '''Функция для математического округления'''

    return int(num + (0.5 if num > 0 else -0.5))


int_r(data.runtime.mean())

110

# 5. Каково медианное значение длительности фильмов? 

In [151]:
# Ответ на вопрос № 5. Верно "+"
answers['5'] = '1. 107'

In [152]:
data.runtime.median()

107.0

In [153]:
# Откинем дробную часть

int(data.runtime.median())

107

# 6. Какой самый прибыльный фильм?
#### Внимание! Здесь и далее под «прибылью» или «убытками» понимается разность между сборами и бюджетом фильма. (прибыль = сборы - бюджет) в нашем датасете это будет (profit = revenue - budget) 

In [154]:
# Ответ на вопрос № 6. Верно "+"
answers['6'] = '5. Avatar (tt0499549)'

In [155]:
data[data.profit == data.profit.max()][['original_title', 'imdb_id', 'budget', 'revenue', 'profit']]

,original_title,imdb_id,budget,revenue,profit
239,Avatar,tt0499549,237000000,2781505847,2544505847


ВАРИАНТ 2

In [156]:
# Вариант 2. Фильтрация через "query"
data.query('profit == {}'.format(data.profit.max()))

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,...,release_date,vote_average,release_year,profit,genre_list,director_list,cast_list,tittle_width,production_company_list,description_width
239,tt0499549,237000000,2781505847,Avatar,Sam Worthington|Zoe Saldana|Sigourney Weaver|S...,James Cameron,Enter the World of Pandora.,"In the 22nd century, a paraplegic Marine is di...",162,Action|Adventure|Fantasy|Science Fiction,...,12/10/2009,7.1,2009,2544505847,"[Action, Adventure, Fantasy, Science Fiction]",[James Cameron],"[Sam Worthington, Zoe Saldana, Sigourney Weave...",6,"[Ingenious Film Partners, Twentieth Century Fo...",28


# 7. Какой фильм самый убыточный? 

In [157]:
# Ответ на вопрос № 7. Верно "+"
answers['7'] = '5. The Lone Ranger (tt1210819)'

In [158]:
data[data.profit == data.profit.min()][['original_title', 'imdb_id', 'budget', 'revenue', 'profit']]

,original_title,imdb_id,budget,revenue,profit
1245,The Lone Ranger,tt1210819,255000000,89289910,-165710090


ВАРИАНТ 2

In [159]:
# Вариант 2. Фильтрация через "query"
data.query('profit == {}'.format(data.profit.min()))

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,...,release_date,vote_average,release_year,profit,genre_list,director_list,cast_list,tittle_width,production_company_list,description_width
1245,tt1210819,255000000,89289910,The Lone Ranger,Johnny Depp|Armie Hammer|William Fichtner|Hele...,Gore Verbinski,Never Take Off the Mask,The Texas Rangers chase down a gang of outlaws...,149,Action|Adventure|Western,...,7/3/2013,6.0,2013,-165710090,"[Action, Adventure, Western]",[Gore Verbinski],"[Johnny Depp, Armie Hammer, William Fichtner, ...",15,"[Walt Disney Pictures, Jerry Bruckheimer Films...",68


# 8. У скольких фильмов из датасета объем сборов оказался выше бюджета?

In [160]:
# Ответ на вопрос № 8. Верно "+"
answers['8'] = '1. 1478'

In [161]:
data[data.profit > 0].count()

imdb_id                    1478
budget                     1478
revenue                    1478
original_title             1478
cast                       1478
director                   1478
tagline                    1478
overview                   1478
runtime                    1478
genres                     1478
production_companies       1478
release_date               1478
vote_average               1478
release_year               1478
profit                     1478
genre_list                 1478
director_list              1478
cast_list                  1478
tittle_width               1478
production_company_list    1478
description_width          1478
dtype: int64

ВАРИАНТ 2

In [162]:
data.loc[data.profit > 0]['imdb_id'].count()

1478

ВАРИАНТ 3

In [163]:
# Так как в DataFrame пустых данных нет, то можно посчитать через длину столбца id
len(data[data.profit > 0].imdb_id.values)

# Либо через индексы тех же id
len(data[data.profit > 0].imdb_id.index)

# Также получилось посчитать вот так
len(data[data.profit > 0])

1478

ВАРИАНТ 4

In [164]:
# Посмотреть в конце получившейся таблице, количество строк
data[data.profit > 0]

# Просто посмотреть информацию
data[data.profit > 0].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1478 entries, 0 to 1884
Data columns (total 21 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   imdb_id                  1478 non-null   object 
 1   budget                   1478 non-null   int64  
 2   revenue                  1478 non-null   int64  
 3   original_title           1478 non-null   object 
 4   cast                     1478 non-null   object 
 5   director                 1478 non-null   object 
 6   tagline                  1478 non-null   object 
 7   overview                 1478 non-null   object 
 8   runtime                  1478 non-null   int64  
 9   genres                   1478 non-null   object 
 10  production_companies     1478 non-null   object 
 11  release_date             1478 non-null   object 
 12  vote_average             1478 non-null   float64
 13  release_year             1478 non-null   int64  
 14  profit                  

# 9. Какой фильм оказался самым кассовым в 2008 году?

In [165]:
# Ответ на вопрос № 9. Верно "+"
answers['9'] = '4. The Dark Knight (tt0468569)'

In [166]:
data[
    (data.release_year == 2008) &
    (data.profit == data[data.release_year == 2008].profit.max())
][[
    'original_title',
    'imdb_id',
    'release_year',
    'profit'
]]

,original_title,imdb_id,release_year,profit
599,The Dark Knight,tt0468569,2008,816921825


# 10. Самый убыточный фильм за период с 2012 по 2014 г. (включительно)?


In [167]:
# Ответ на вопрос № 10. Верно "+"
answers['10'] = '5. The Lone Ranger (tt1210819)'

In [216]:
data[
    (data.release_year >= 2012) &
    (data.release_year <= 2014) &
    (data.profit == data[(data.release_year > 2012) & (data.release_year <= 2014)].profit.min())
][[
    'original_title',
    'imdb_id',
    'release_year',
    'profit'
]]

,original_title,imdb_id,release_year,profit
1245,The Lone Ranger,tt1210819,2013,-165710090


# 11. Какого жанра фильмов больше всего?

In [169]:
# Ответ на вопрос № 11. Верно "+"
answers['11'] = '3. Drama'

In [170]:
genres_data = data.genres.values
pd.Series('|'.join(genres_data).split('|')).value_counts()

Drama              782
Comedy             683
Thriller           596
Action             582
Adventure          415
Crime              315
Romance            308
Family             260
Science Fiction    248
Fantasy            222
Horror             176
Mystery            168
Animation          139
Music               64
History             62
War                 58
Western             19
Documentary          8
Foreign              2
dtype: int64

ВАРИАНТ 2

In [171]:
# Сделаем новый DataFrame
genre_list_data = data.explode('genre_list')

genre_list_data.genre_list.value_counts()

Drama              782
Comedy             683
Thriller           596
Action             582
Adventure          415
Crime              315
Romance            308
Family             260
Science Fiction    248
Fantasy            222
Horror             176
Mystery            168
Animation          139
Music               64
History             62
War                 58
Western             19
Documentary          8
Foreign              2
Name: genre_list, dtype: int64

ВАРИАНТ 3

In [172]:
# Можно сделать через цикл for, но думаю этого делать не стоит)))
# На одном из вебиранов докладчик, говорил, что обработка данных
    #посредством циклов очень медленная, поэтому лучше примениять библотеке

# 12. Фильмы какого жанра чаще всего становятся прибыльными? 

In [173]:
# Ответ на вопрос № 12. Верно "+"
answers['12'] = '1. Drama'

In [174]:
genres_data = data[data.profit > 0].genres.values
gender_data_rating = pd.Series('|'.join(genres_data).split('|')).value_counts()

gender_data_rating

Drama              560
Comedy             551
Thriller           446
Action             444
Adventure          337
Romance            242
Crime              231
Family             226
Science Fiction    195
Fantasy            188
Horror             150
Animation          120
Mystery            119
Music               47
History             46
War                 41
Western             12
Documentary          7
dtype: int64

# 13. У какого режиссера самые большие суммарные кассовые сборы?

In [175]:
# Ответ на вопрос № 13. Верно "+"
answers['13'] = '5. Peter Jackson'

In [176]:
director_list_data = data.explode('director_list')

director_revenue_data = director_list_data.pivot_table(
        values=['revenue'],
        index=['director_list'],
        aggfunc='sum',
        fill_value = 'director'
    ).sort_values(
        ['revenue'],
        ascending=False
    )

# director_revenue_data.iloc[0]
director_revenue_data

,revenue
director_list,
Peter Jackson,6490593685
Christopher Nolan,4167548502
David Yates,4154295625
Michael Bay,3886938960
J.J. Abrams,3579169916
...,...
David MichÃ´d,2295423
Steven Shainberg,2281089
Paul Schrader,2062066


# 14. Какой режисер снял больше всего фильмов в стиле Action?

In [177]:
# Ответ на вопрос № 14. Верно "+"
answers['14'] = '3. Robert Rodriguez'

In [178]:
director_list_data = data.explode('director_list')

director_list_data[director_list_data.genres.str.contains('Action', na=False)].director_list.value_counts()

Robert Rodriguez      9
Paul W.S. Anderson    7
Michael Bay           7
Ridley Scott          6
Antoine Fuqua         6
                     ..
James Mather          1
Mikael HÃ¥fstrÃ¶m     1
Robert Redford        1
Steven Brill          1
Randall Wallace       1
Name: director_list, Length: 364, dtype: int64

ВАРИАНТ 2

In [179]:
genre_list_data = data.explode('genre_list')
director_genre_list_data = genre_list_data.explode('director_list')

director_genre_list_data[director_genre_list_data.genre_list == 'Action']['director_list'].value_counts()

Robert Rodriguez      9
Paul W.S. Anderson    7
Michael Bay           7
Ridley Scott          6
Antoine Fuqua         6
                     ..
James Mather          1
Mikael HÃ¥fstrÃ¶m     1
Robert Redford        1
Steven Brill          1
Randall Wallace       1
Name: director_list, Length: 364, dtype: int64

ВАРИАНТ 3

In [180]:
genre_list_data = data.explode('genre_list')
director_genre_list_data = genre_list_data.explode('director_list')

director_genre = director_genre_list_data.loc[
        director_genre_list_data[
            'genre_list'
        ].isin([
            'Action'
        ])
    ].pivot_table(
        values=['imdb_id'],
        index=['director_list'],
        columns=['genre_list'],
        aggfunc='count'
    )

director_genre_data = director_genre.reset_index()
director_genre_data.columns = ['director', 'action']

director_genre_data.sort_values(
        ['action'],
        ascending=False
    )

,director,action
296,Robert Rodriguez,9
231,Michael Bay,7
261,Paul W.S. Anderson,7
286,Ridley Scott,6
23,Antoine Fuqua,6
...,...,...
178,John Whitesell,1
42,Brian A Miller,1
180,Jon Amiel,1
181,Jon Avnet,1


# 15. Фильмы с каким актером принесли самые высокие кассовые сборы в 2012 году? 

In [181]:
# Ответ на вопрос № 15. Верно "+"
answers['15'] = '3. Chris Hemsworth'

In [182]:
cast_list_data = data[data.release_year == 2012].explode('cast_list')

cast_revenue = cast_list_data.pivot_table(
        values=['revenue'],
        index=['cast_list'],
        aggfunc='sum'
    ).sort_values(
        ['revenue'],
        ascending=False
    )

cast_revenue

,revenue
cast_list,
Chris Hemsworth,2027450773
Denis Leary,1629460639
Anne Hathaway,1522851057
Robert Downey Jr.,1519557910
Mark Ruffalo,1519557910
...,...
Michael Nyqvist,3428048
Danny Huston,2106557
Josh Lucas,2106557


ВАРИАНТ 2

In [183]:
# Не нравится мне конечно, что тут функция обращается к внешней переменной
    # Возможно было бы правильней data_2012 передать функции get_revenue, как параметр по умолчанию

%time

data_2012 = data[data.release_year == 2012]


def get_revenue(name):
    '''Функция вычисляет сумму кассовых сборов
        по имени актера'''

    return data_2012[data_2012.cast.str.contains(name, na=False)]['revenue'].sum()


cast_data = data_2012[data_2012.release_year == 2012]['cast'].values
cast_data_series = pd.Series('|'.join(cast_data).split('|')).drop_duplicates()
cast_data_series_list = cast_data_series.values

cast_data_df = pd.DataFrame(cast_data_series_list)
cast_data_df.columns = ['name']

cast_data_df['revenue'] = cast_data_df.name.apply(get_revenue)

cast_data_df.sort_values(
        ['revenue'],
        ascending=False
    )

Wall time: 0 ns


,name,revenue
3,Chris Hemsworth,2027450773
48,Denis Leary,1629460639
13,Anne Hathaway,1522851057
0,Robert Downey Jr.,1519557910
2,Mark Ruffalo,1519557910
...,...,...
327,Jason Bateman,3428048
272,Nicolas Cage,2106557
275,Sami Gayle,2106557
274,Josh Lucas,2106557


# 16. Какой актер снялся в большем количестве высокобюджетных фильмов?

In [184]:
# Ответ на вопрос № 16. Верно "+"
answers['16'] = '3. Matt Damon'

In [185]:
cast_data = data[data.budget > data.budget.mean()]['cast'].values
cast_data_series = pd.Series('|'.join(cast_data).split('|')).value_counts()

series_cast = cast_data_series.reset_index()

series_cast['index'][0]
# series_cast

'Matt Damon'

# 17. В фильмах какого жанра больше всего снимался Nicolas Cage? 

In [186]:
# Ответ на вопрос № 17. Верно "+"
answers['17'] = '2. Action'

In [187]:
genres_data = data[data.cast.str.contains('Nicolas Cage')]['genres'].values
gender_data_rating = pd.Series('|'.join(genres_data).split('|')).value_counts()

gender_data_rating

Action             17
Thriller           15
Drama              12
Crime              10
Fantasy             8
Adventure           7
Comedy              6
Science Fiction     4
Family              3
Mystery             3
Animation           3
History             2
War                 1
Horror              1
Romance             1
dtype: int64

# 18. Самый убыточный фильм от Paramount Pictures

In [188]:
# Ответ на вопрос № 18. Верно "+"
answers['18'] = '1. K-19: The Widowmaker (tt0267626)'

In [189]:
# Ниже тоже самое написано более человекопонятным синтаксисом
    # На мой взгляд не всегда стоит гнаться за "меньшим количеством строчек кода")))
# data[(data.production_companies.str.contains('Paramount Pictures')) & (data.profit == data[data.production_companies.str.contains('Paramount Pictures')]['profit'].min())]

min_profit = data[data.production_companies.str.contains('Paramount Pictures')]['profit'].min()
data[(data.production_companies.str.contains('Paramount Pictures')) & (data.profit == min_profit)]

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,...,release_date,vote_average,release_year,profit,genre_list,director_list,cast_list,tittle_width,production_company_list,description_width
925,tt0267626,100000000,35168966,K-19: The Widowmaker,Harrison Ford|Liam Neeson|Peter Sarsgaard|Joss...,Kathryn Bigelow,Fate has found its hero.,When Russia's first nuclear submarine malfunct...,138,Thriller|Drama|History,...,7/19/2002,6.0,2002,-64831034,"[Thriller, Drama, History]",[Kathryn Bigelow],"[Harrison Ford, Liam Neeson, Peter Sarsgaard, ...",20,"[Paramount Pictures, Intermedia Films, Nationa...",23


# 19. Какой год стал самым успешным по суммарным кассовым сборам?

In [190]:
# Ответ на вопрос № 19. Верно "+"
answers['19'] = '5. 2015'

In [191]:
data.pivot_table(
        values=['revenue'],
        index=['release_year'],
        aggfunc='sum',
        fill_value = 0
    ).sort_values(
        ['revenue'],
        ascending=False
    )

,revenue
release_year,
2015,25449202382
2014,23405862953
2013,23213799791
2012,23079001687
2011,22676791872
2010,21071204961
2009,20261791024
2008,18252781990
2007,18162406801


In [192]:
successful_year = data.loc[
        data[
            'release_year'
        ].isin([
            '2014',
            '2008',
            '2012',
            '2002',
            '2015'
        ])
    ].pivot_table(
        values=['revenue'],
        index=['release_year'],
        aggfunc='sum',
        fill_value = 0
    ).sort_values(
        ['revenue'],
        ascending=False
    )

successful_year

,revenue
release_year,
2015,25449202382
2014,23405862953
2012,23079001687
2008,18252781990
2002,14136361487


# 20. Какой самый прибыльный год для студии Warner Bros?

In [193]:
# Ответ на вопрос № 20. Верно "+"
answers['20'] = '1. 2014'

In [194]:
data_filter_wb = data[data.production_companies.str.contains('Warner Bros')]

successful_year = data_filter_wb.pivot_table(
        values=['profit'],
        index=['release_year'],
        aggfunc='sum',
        fill_value = 0
    ).sort_values(
        ['profit'],
        ascending=False
    )

successful_year

,profit
release_year,
2014,2295464519
2007,2201675217
2008,2134595031
2010,1974712985
2011,1871393682
2003,1855493377
2009,1822454136
2013,1636453400
2004,1631933725


In [195]:
data_filter_wb = data[data.production_companies.str.contains('Warner Bros')]

successful_year = data_filter_wb.loc[
        data_filter_wb[
            'release_year'
        ].isin([
            '2014',
            '2008',
            '2012',
            '2010',
            '2015'
        ])
    ].pivot_table(
        values=['profit'],
        index=['release_year'],
        aggfunc='sum',
        fill_value = 0
    ).sort_values(
        ['profit'],
        ascending=False
    )

successful_year

,profit
release_year,
2014,2295464519
2008,2134595031
2010,1974712985
2012,1258020056
2015,870368348


# 21. В каком месяце за все годы суммарно вышло больше всего фильмов?

In [196]:
# Ответ на вопрос № 21. Верно "+"
answers['21'] = '4. Сентябрь'

In [197]:
month_data = pd.DataFrame({'count_film': [
    len(data[data.release_date.str.match('1/')]),
    len(data[data.release_date.str.match('2/')]),
    len(data[data.release_date.str.match('3/')]),
    len(data[data.release_date.str.match('4/')]),
    len(data[data.release_date.str.match('5/')]),
    len(data[data.release_date.str.match('6/')]),
    len(data[data.release_date.str.match('7/')]),
    len(data[data.release_date.str.match('8/')]),
    len(data[data.release_date.str.match('9/')]),
    len(data[data.release_date.str.match('10/')]),
    len(data[data.release_date.str.match('11/')]),
    len(data[data.release_date.str.match('12/')]),
]}, index=['Январь', 'Февраль', 'Март', 'Апрель', 'Май', 'Июнь', 'Июль', 'Август', 'Сентябрь', 'Октябрь', 'Ноябрь', 'Декабрь'])

month_data.sort_values(
        ['count_film'],
        ascending=False
    )

,count_film
Сентябрь,227
Декабрь,190
Октябрь,186
Август,161
Март,156
Апрель,149
Июнь,147
Ноябрь,146
Июль,142
Май,140


# 22. Сколько суммарно вышло фильмов летом? (за июнь, июль, август)

In [198]:
# Ответ на вопрос № 22. Верно "+"
answers['22'] = '2. 450'

In [199]:
month_data = pd.Series([
    len(data[data.release_date.str.match('6/')]),
    len(data[data.release_date.str.match('7/')]),
    len(data[data.release_date.str.match('8/')]),
]).sum()

month_data

450

# 23. Для какого режиссера зима – самое продуктивное время года? 

In [200]:
# Ответ на вопрос № 23. Верно "+"
answers['23'] = '5. Peter Jackson'

In [201]:
data[
    data.release_date.str.match('12/') |
    data.release_date.str.match('1/') |
    data.release_date.str.match('2/')
].explode('director_list').director_list.value_counts()

Peter Jackson         7
Clint Eastwood        6
Steven Soderbergh     6
Martin Scorsese       4
Shawn Levy            4
                     ..
Tom Tykwer            1
Alan Parker           1
Kelly Asbury          1
Sam Taylor-Johnson    1
David Fincher         1
Name: director_list, Length: 358, dtype: int64

# 24. Какая студия дает самые длинные названия своим фильмам по количеству символов?

In [202]:
# Ответ на вопрос № 24. Верно "+"
answers['24'] = '5. Four By Two Productions'

In [203]:
# В выборке всего один фильм от этой компании, но у него длинное название
    # Как мне кажется такие экстремальные значения надо выкидывать из анализа)))

production_companies_data = data.explode('production_company_list')

pivot_production_companies_data = production_companies_data.loc[
        production_companies_data[
            'production_company_list'
        ].isin([
            'Universal Pictures',
            'Universal',
            'Warner Bros',
            'Jim Henson Company, The',
            'Paramount Pictures',
            'Four By Two Productions'
        ])
    ].pivot_table(
        values=['tittle_width'],
        index=['production_company_list'],
        aggfunc='mean',
        fill_value = 0
    ).sort_values(
        ['tittle_width'],
        ascending=False
    )

pivot_production_companies_data

,tittle_width
production_company_list,
Four By Two Productions,83.000000
"Jim Henson Company, The",59.000000
Paramount Pictures,17.032787
Universal Pictures,14.716763
Universal,13.333333


# 25. Описание фильмов какой студии в среднем самые длинные по количеству слов?

In [204]:
# Ответ на вопрос № 25. Верно "+"
answers['25'] = '3. Midnight Picture Show'

In [205]:
production_companies_data = data.explode('production_company_list')

production_companies_description_data = production_companies_data.loc[
        production_companies_data[
            'production_company_list'
        ].isin([
            'Universal Pictures',
            'Universal',
            'Warner Bros',
            'Midnight Picture Show',
            'Paramount Pictures',
            'Total Entertainment'
        ])
    ].pivot_table(
        values=['description_width'],
        index=['production_company_list'],
        aggfunc='mean',
        fill_value = 0
    ).sort_values(
        ['description_width'],
        ascending=False
    )

production_companies_description_data

,description_width
production_company_list,
Midnight Picture Show,175.000000
Total Entertainment,86.000000
Paramount Pictures,55.278689
Universal Pictures,53.682081
Universal,44.666667


# 26. Какие фильмы входят в 1 процент лучших по рейтингу? 
по vote_average

In [206]:
# Ответ на вопрос № 26. Верно "+"
answers['26'] = '1. Inside Out, The Dark Knight, 12 Years a Slave'

In [215]:
vote_average_data = data.pivot_table(
        values=['vote_average'],
        index=['original_title'],
        fill_value = 0
    ).sort_values(
        ['vote_average'],
        ascending=False
    )


def int_r(num):
    '''Функция для математического округления'''

    return int(num + (0.5 if num > 0 else -0.5))


vote_average_data.head(int_r(len(data['original_title']) / 100))

,vote_average
original_title,
The Dark Knight,8.1
The Imitation Game,8.0
Interstellar,8.0
Room,8.0
Inside Out,8.0
Memento,7.9
The Pianist,7.9
Inception,7.9
The Grand Budapest Hotel,7.9


# 27. Какие актеры чаще всего снимаются в одном фильме вместе?


In [208]:
# Ответ на вопрос № 27. Верно "+"
answers['27'] = '5. Daniel Radcliffe & Rupert Grint'

In [209]:
dict_actors_couple = {
    'Johnny Depp & Helena Bonham Carter': len(
        data[
            (data.cast.str.contains('Johnny Depp')) &
            (data.cast.str.contains('Helena Bonham Carter'))
        ]['original_title']),
    'Ben Stiller & Owen Wilson': len(
        data[
            (data.cast.str.contains('Ben Stiller')) &
            (data.cast.str.contains('Owen Wilson'))
        ]['original_title']),
    'Vin Diesel & Paul Walker': len(
        data[
            (data.cast.str.contains('Vin Diesel')) &
            (data.cast.str.contains('Paul Walker'))
        ]['original_title']),
    'Adam Sandler & Kevin James': len(
        data[
            (data.cast.str.contains('Adam Sandler')) &
            (data.cast.str.contains('Kevin James'))
        ]['original_title']),
    'Daniel Radcliffe & Rupert Grint': len(
        data[
            (data.cast.str.contains('Daniel Radcliffe')) &
            (data.cast.str.contains('Rupert Grint'))
        ]['original_title']),
}

dict_actors_couple

{'Johnny Depp & Helena Bonham Carter': 6,
 'Ben Stiller & Owen Wilson': 6,
 'Vin Diesel & Paul Walker': 5,
 'Adam Sandler & Kevin James': 5,
 'Daniel Radcliffe & Rupert Grint': 8}

# Submission

In [210]:
# в конце можно посмотреть свои ответы к каждому вопросу
answers

{'1': '5. Pirates of the Caribbean: On Stranger Tides (tt1298650)',
 '2': '2. Gods and Generals (tt0279111)',
 '3': '3. Winnie the Pooh (tt1449283)',
 '4': '2. 110',
 '5': '1. 107',
 '6': '5. Avatar (tt0499549)',
 '7': '5. The Lone Ranger (tt1210819)',
 '8': '1. 1478',
 '9': '4. The Dark Knight (tt0468569)',
 '10': '5. The Lone Ranger (tt1210819)',
 '11': '3. Drama',
 '12': '1. Drama',
 '13': '5. Peter Jackson',
 '14': '3. Robert Rodriguez',
 '15': '3. Chris Hemsworth',
 '16': '3. Matt Damon',
 '17': '2. Action',
 '18': '1. K-19: The Widowmaker (tt0267626)',
 '19': '5. 2015',
 '20': '1. 2014',
 '21': '4. Сентябрь',
 '22': '2. 450',
 '23': '5. Peter Jackson',
 '24': '5. Four By Two Productions',
 '25': '3. Midnight Picture Show',
 '26': '1. Inside Out, The Dark Knight, 12 Years a Slave',
 '27': '5. Daniel Radcliffe & Rupert Grint'}

In [211]:
# и убедиться что ни чего не пропустил)
len(answers)

27

In [212]:
# Многое из того, что подключено в начале (библиотечки) не использовал..., обошелся уже проейденными знаниями

# Некоторые задачи можно было сделать с помощью циклов, но я не стал даже думать в эту сторону
    # Так как на одном из вебинаров было сказано, что циклы для перебора данных слишком медленные(((

# Рад, что успел и всё получилось... К сожалению ноябрь месяц, был напряженный на работе, поэтому ноутбук по
    # учебе удалось заполнить только к дедлайну(, но думаю в следующих месяцах исправлюсь))))

In [0]:
# Спасибо!) Во время выполнения проекта, получил хороший опыт!)))